In [1]:
import time
import pandas as pd
import requests
import json5

# Loading national data

In [2]:
df_nat = pd.read_csv("../Data/Employment_Projections.csv").sort_values('Employment 2030',ascending=False)

# Loading CA data

In [6]:
df_CA = pd.read_csv("../Data/CA_Long_Term_Occupational_Employment_Projections.csv").sort_values('Projected Year Employment Estimate',ascending=False)

In [9]:
df_Sac = df_CA[df_CA['Area Name (County Names)']=='Sacramento--Roseville--Arden-Arcade MSA (El Dorado, Placer, Sacramento, and Yolo Counties)'].copy()

In [11]:
df_Cal = df_CA[df_CA['Area Name (County Names)']=='California'].copy()

Filtering for those occupations that make 40k a year or more and cleaning occupational code for the national table to match the california tables

In [13]:
df_Sac = df_Sac[df_Sac['Median Annual Wage']>=40000].copy()

In [14]:
df_nat['Occupation Code']=df_nat['Occupation Code'].str.extract(r'([0-9]{2}-[0-9]{4})')

need to bin education levels

In [15]:
df_Sac['Entry Level Education'].value_counts()

High school diploma or equivalent    108
Bachelor's degree                    103
Doctoral or professional degree       28
Associate's degree                    24
Postsecondary non-degree award        19
No formal educational credential      18
Master's degree                       18
Some college, no degree                3
Name: Entry Level Education, dtype: int64

In [18]:
education_levels = {'No formal educational credential':'<HS',
                    'High school diploma or equivalent':'HS+',
                    "Bachelor's degree":'Associates+',
                    "Associate's degree":'Associates+',
                    'Postsecondary non-degree award':'HS+',
                    'Some college, no degree':'HS+'
    
}
df_Sac['Education bin'] = df_Sac['Entry Level Education'].replace(to_replace=education_levels)
df_Cal['Education bin'] = df_Cal['Entry Level Education'].replace(to_replace=education_levels)

In [22]:
df_Sac[df_Sac['Education bin']=='<HS'].sort_values(by='Projected Year Employment Estimate',ascending=False).head()

,Area Type,Area Name (County Names),Period,SOC Level,Standard Occupational Classification (SOC),Occupational Title,Base Year Employment Estimate,Projected Year Employment Estimate,Numeric Change,Percentage Change,Exits,Transfers,Total Job Openings,Median Hourly Wage,Median Annual Wage,Entry Level Education,Work Experience,Job Training,Education bin
9742,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,47-2061,Construction Laborers,11240,12210.0,970,8.6,3920,9120,14010,21.65,45048.0,No formal educational credential,None,Short-term on-the-job training,<HS
9749,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,47-2141,"Painters, Construction and Maintenance",3400,3860.0,460,13.5,1260,2420,4140,22.44,46666.0,No formal educational credential,None,Moderate-term on-the-job training,<HS
9890,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,53-7051,Industrial Truck and Tractor Operators,2200,2490.0,290,13.2,700,1940,2930,19.91,41415.0,No formal educational credential,None,Short-term on-the-job training,<HS
9745,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,47-2081,Drywall and Ceiling Tile Installers,2270,2480.0,210,9.3,700,1670,2580,24.38,50704.0,No formal educational credential,None,Moderate-term on-the-job training,<HS
9741,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,47-2051,Cement Masons and Concrete Finishers,2010,2190.0,180,9.0,610,1590,2380,26.22,54536.0,No formal educational credential,None,Moderate-term on-the-job training,<HS


In [23]:
df_Sac[df_Sac['Education bin']=='HS+'].sort_values(by='Projected Year Employment Estimate',ascending=False).head()

,Area Type,Area Name (County Names),Period,SOC Level,Standard Occupational Classification (SOC),Occupational Title,Base Year Employment Estimate,Projected Year Employment Estimate,Numeric Change,Percentage Change,Exits,Transfers,Total Job Openings,Median Hourly Wage,Median Annual Wage,Entry Level Education,Work Experience,Job Training,Education bin
9738,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,47-2031,Carpenters,14310,15920.0,1610,11.3,4730,10920,17260,27.27,56730.0,High school diploma or equivalent,None,Apprenticeship,HS+
9713,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,43-6014,"Secretaries and Administrative Assistants, Exc...",14180,13300.0,-880,-6.2,7040,8340,14500,20.13,41851.0,High school diploma or equivalent,None,Short-term on-the-job training,HS+
9675,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,43-3031,"Bookkeeping, Accounting, and Auditing Clerks",10550,10230.0,-320,-3.0,6320,5780,11780,22.04,45845.0,"Some college, no degree",None,Moderate-term on-the-job training,HS+
9669,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,43-1011,First-Line Supervisors of Office and Administr...,9660,9730.0,70,0.7,3630,6520,10220,29.77,61936.0,High school diploma or equivalent,<5 years,None,HS+
9642,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,41-1011,First-Line Supervisors of Retail Sales Workers,9650,9700.0,50,0.5,3340,7290,10680,21.86,45464.0,High school diploma or equivalent,<5 years,None,HS+


In [24]:
df_Sac[df_Sac['Education bin']=='Associates+'].sort_values(by='Projected Year Employment Estimate',ascending=False).head()

,Area Type,Area Name (County Names),Period,SOC Level,Standard Occupational Classification (SOC),Occupational Title,Base Year Employment Estimate,Projected Year Employment Estimate,Numeric Change,Percentage Change,Exits,Transfers,Total Job Openings,Median Hourly Wage,Median Annual Wage,Entry Level Education,Work Experience,Job Training,Education bin
9307,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,13-1111,Management Analysts,27320,31200.0,3880,14.2,9080,18480,31440,37.15,77276.0,Bachelor's degree,<5 years,None,Associates+
9523,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,29-1141,Registered Nurses,18890,22360.0,3470,18.4,5720,5290,14480,64.13,133381.0,Bachelor's degree,None,None,Associates+
9266,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,11-1021,General and Operations Managers,14660,15880.0,1220,8.3,3200,10060,14480,51.72,107586.0,Bachelor's degree,≥5 years,None,Associates+
9315,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,13-2011,Accountants and Auditors,12000,12350.0,350,2.9,3440,7910,11700,37.15,77286.0,Bachelor's degree,None,None,Associates+
9460,Metropolitan Area,Sacramento--Roseville--Arden-Arcade MSA (El Do...,2018-2028,4,25-2021,"Elementary School Teachers, Except Special Edu...",8720,9060.0,340,3.9,2770,3790,6900,0.00,72360.0,Bachelor's degree,None,None,Associates+
